1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
3. Построить случайный лес, вывести важность признаков
4. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
5. Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
6. Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
7. В качестве решения: Jupyter notebook с кодом, комментариями и графиками, ссылка на гитхаб

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import auc, roc_curve, roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv('train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
#data.info()

In [4]:
#в описание датасета сказано, что  в категриальных данных, nan - это не пропуски, а значения Not Available. Заполняю nan занчениями NA.
data = data.fillna('NA')

In [5]:
data.describe()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,10516.828082,6.099315,5.575342,1971.267808,1984.865753,443.639726,46.549315,567.240411,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,9981.264932,1.382997,1.112799,30.202904,20.645407,456.098091,161.319273,441.866955,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,223.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,383.500000,0.000000,477.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,712.250000,0.000000,808.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,5644.000000,1474.000000,2336.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [6]:
#для классификации создаю категириальные данные из 'SalePrice' по квантилям
quant = []

qunt_list = [0.25, 0.5, 0.75]

for i in qunt_list:
    quant.append(int(np.quantile(data.SalePrice, i)))
print(quant)

#почемуто этот код не сработал:
#for i in data['SalePrice']:
#    if i < quant[0]:
#        data['SalePrice_cls'] = 'SalePrice < 0.25q'
#    elif i in range(quant[0], quant[1]):
#        data['SalePrice_cls'] = 'SalePrice in range (0.25, 0.5)q'
#    elif i in range(quant[1], quant[2]):
#            data['SalePrice_cls'] = 'SalePrice in range (0.5, 0.75)q'
#    elif i > quant[2]:
#        data['SalePrice_cls'] = 'SalePrice in range > 0.75q'

[129975, 163000, 214000]


In [7]:
sp_clf025 = data.loc[data['SalePrice'] <= quant[0], ['SalePrice']].index
sp_clf025_05 = data.loc[(data['SalePrice'] > quant[0]) & (data['SalePrice'] <= quant[1]), ['SalePrice']].index
sp_clf05_075 = data.loc[(data['SalePrice'] > quant[1]) & (data['SalePrice'] < quant[2]), ['SalePrice']].index
sp_clf075 = data.loc[data['SalePrice'] >= quant[2], ['SalePrice']].index

data['SalePrice_cls'] = None
data['SalePrice_cls'][sp_clf025] = 'SalePrice < 0.25q'
data['SalePrice_cls'][sp_clf025_05] = 'SalePrice in range (0.25, 0.5)q'
data['SalePrice_cls'][sp_clf05_075] = 'SalePrice in range (0.5, 0.75)q'
data['SalePrice_cls'][sp_clf075] = 'SalePrice in range > 0.75q'

data['SalePrice_cls'].unique()


D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
D:\Program_Files\Anaconda\lib\site-packages\ipykernel_laun

array(['SalePrice in range (0.5, 0.75)q', 'SalePrice in range > 0.75q',
       'SalePrice in range (0.25, 0.5)q', 'SalePrice < 0.25q'],
      dtype=object)

In [8]:
data['SalePrice_cls'].describe()

count                           1460
unique                             4
top       SalePrice in range > 0.75q
freq                             367
Name: SalePrice_cls, dtype: object

In [9]:
#Код data.fillna('NA') заполнил пропуски во всех столбцах, вkлючая float64
#тут создаю список категориалных данных и удалю из него данные типа float
cat_list = list(data.dtypes[data.dtypes == object].index)
float_list = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

for i in float_list:
    cat_list.remove(i)

#cat_list

In [10]:
# удаляю NA из столбцов типа float64 - заполняю медианными значениями
med = data.loc[data['LotFrontage'] != 'NA'].median().LotFrontage
data.loc[data['LotFrontage'] == 'NA', ['LotFrontage']] = med
data['LotFrontage'] = data['LotFrontage'].astype(int)

med = data.loc[data['MasVnrArea'] != 'NA'].median().MasVnrArea
data.loc[data['MasVnrArea'] == 'NA', ['MasVnrArea']] = med
data['MasVnrArea'] = data['MasVnrArea'].astype(int)

med = int(data.loc[data['GarageYrBlt'] != 'NA'].median().GarageYrBlt)
data.loc[data['GarageYrBlt'] == 'NA', ['GarageYrBlt']] = med
data['GarageYrBlt'] = data['GarageYrBlt'].astype(int)

In [11]:
# удаляю признаки, где уникальных значений >10
cat_nunique = data[cat_list].nunique()
cat_feat10 = list(cat_nunique[cat_nunique > 10].index)

data[cat_feat10].head()

,Neighborhood,Exterior1st,Exterior2nd
0,CollgCr,VinylSd,VinylSd
1,Veenker,MetalSd,MetalSd
2,CollgCr,VinylSd,VinylSd
3,Crawfor,Wd Sdng,Wd Shng
4,NoRidge,VinylSd,VinylSd


In [12]:
# делю датафреймн на данные и таргет
# выделять наиболее важные признаки не буду, однако это можно сделать: проанализировав данные, 
#удалив категориальные фичи у которых число уникальных значений большое (>10)

drop_list = cat_feat10 + ['Id', 'SalePrice'] #'Id' точно не нужeн. Датафрейм содержит категориальные признаки

X = data.drop(drop_list, axis = 'columns')
Y = data['SalePrice_cls']

#drop_list
#cat_list
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice_cls
0,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,2,2008,WD,Normal,"SalePrice in range (0.5, 0.75)q"
1,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,5,2007,WD,Normal,"SalePrice in range (0.5, 0.75)q"
2,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,9,2008,WD,Normal,SalePrice in range > 0.75q
3,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,"SalePrice in range (0.25, 0.5)q"
4,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,12,2008,WD,Normal,SalePrice in range > 0.75q


### 3. Построить случайный лес, вывести важность признаков

In [13]:
#преобразоване категориальных данных

from sklearn.preprocessing import LabelEncoder

le_X = LabelEncoder()
le_Y = LabelEncoder()

#выделяю катеогориалые данные
X_clf = data[cat_list]
X_clf.drop(columns = ['SalePrice_cls'], axis = 1, inplace = True)

for i in X_clf:
    X_clf[i] = le_X.fit_transform(X_clf[i])


Y_clf = le_Y.fit_transform(Y)

D:\Program_Files\Anaconda\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
D:\Program_Files\Anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [14]:
X_clf.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,3,1,1,3,3,0,4,0,5,2,...,1,2,5,5,2,3,4,1,8,4
1,3,1,1,3,3,0,2,0,24,1,...,1,2,5,5,2,3,4,1,8,4
2,3,1,1,0,3,0,4,0,5,2,...,1,2,5,5,2,3,4,1,8,4
3,3,1,1,0,3,0,0,0,6,2,...,5,3,5,5,2,3,4,1,8,0
4,3,1,1,0,3,0,2,0,15,2,...,1,2,5,5,2,3,4,1,8,4


In [15]:
# Случайный лес
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier(random_state = 700, n_jobs = -1)

In [16]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10, random_state= 500)
kf.get_n_splits(X_clf)

print(kf)

for train_index, test_index in kf.split(X_clf):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train_clf = X_clf.iloc[train_index]
    X_test_clf = X_clf.iloc[test_index]
    y_train_clf = Y_clf[train_index]
    y_test_clf = Y_clf[test_index]

KFold(n_splits=10, random_state=500, shuffle=False)


In [17]:
scores = []

cv = KFold(n_splits = 10, random_state = 100, shuffle = True)

for train_index, test_index in cv.split(X_clf):
    X_train_clf = X_clf.iloc[train_index]
    X_test_clf = X_clf.iloc[test_index]
    y_train_clf = Y_clf[train_index]
    y_test_clf = Y_clf[test_index]
    
    clf_rf.fit(X_train_clf, y_train_clf)
    scores.append(clf_rf.score(X_test_clf, y_test_clf))
scores

D:\Program_Files\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


[0.6712328767123288,
 0.6164383561643836,
 0.5958904109589042,
 0.6301369863013698,
 0.6164383561643836,
 0.5821917808219178,
 0.6986301369863014,
 0.6438356164383562,
 0.6164383561643836,
 0.6506849315068494]

In [18]:
importances = clf_rf.feature_importances_

In [19]:
# График важности feature importances

indices = np.argsort(importances)[::-1]

import matplotlib.pyplot as plt
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_clf.shape[1]), importances[indices], color="r", align="center")
plt.xlim([-1, X_clf.shape[1]])
plt.show()

<Figure size 640x480 with 1 Axes>

In [20]:
print("Feature importances")
for i in range(len(X_clf.columns)):
    print(X_clf.columns[i], ':', importances[i])

Feature importances
MSZoning : 0.024886282736356912
Street : 0.0
Alley : 0.009359529433098862
LotShape : 0.02284275452245773
LandContour : 0.015633754792024824
Utilities : 0.0
LotConfig : 0.023968799655517407
LandSlope : 0.005556058873476121
Neighborhood : 0.07013025073913876
Condition1 : 0.016635208510301622
Condition2 : 0.0010721252587068324
BldgType : 0.027635032618040588
HouseStyle : 0.036892949867120044
RoofStyle : 0.018307893492948518
RoofMatl : 0.002165591604739787
Exterior1st : 0.032340534646199244
Exterior2nd : 0.0419547963318301
MasVnrType : 0.040104627552453316
ExterQual : 0.04567579102409287
ExterCond : 0.012438434130792233
Foundation : 0.0440035045794392
BsmtQual : 0.06131080686457204
BsmtCond : 0.008343814673587397
BsmtExposure : 0.032568980420280405
BsmtFinType1 : 0.04225087989125053
BsmtFinType2 : 0.016524275603364794
Heating : 0.0013271088162612706
HeatingQC : 0.02484472712399961
CentralAir : 0.008810438124199375
Electrical : 0.0069658234885543145
KitchenQual : 0.04850

Предпологаю, что если в дальнейшем потребуется улучшить качество можели, то можно основыватся на importances и удалить малозначимые фичи

In [21]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

n_samples = 10
cv = ShuffleSplit(n_splits= n_samples, test_size=0.2, random_state=0)
cross_val_score(clf_rf, X_clf, Y_clf, cv = cv, n_jobs = -1)

array([0.63013699, 0.63356164, 0.71917808, 0.63013699, 0.65410959,
       0.6609589 , 0.65410959, 0.69178082, 0.60273973, 0.6369863 ])

### 4. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную

#### Логистическая регрессия

In [22]:
#отфильтруем непрерывные признаки. ['ID'] - синтетический признак он не понадобится
num_list = [f for f in data if f not in (cat_list)]
num_list.remove('Id')
num_list.remove('SalePrice')
#len(num_list)

In [23]:
X_lin = data[num_list]
Y_lin = data['SalePrice']
X_lin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 36 columns):
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null int32
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null int32
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
Fireplaces       1460 non-null int64
Garag

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train_lin, X_test_lin, y_train_lin, y_test_lin = train_test_split(X_lin, Y_lin, test_size = 0.2, random_state = 10)

In [25]:
scaler = StandardScaler()
scaler.fit(X_train_lin[num_list])

X_train_lin = scaler.transform(X_train_lin[num_list])
X_test_lin = scaler.transform(X_test_lin[num_list])

In [26]:
from sklearn.linear_model import LinearRegression
reg_lr = reg = LinearRegression(n_jobs = -1)

reg_lr.fit(X_train_lin, y_train_lin)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [27]:
#y_pred_lin_test = reg_lr.predict(X_test_lin)
#y_pred_lin_train = reg_lr.predict(X_train_lin)

print('Train', reg_lr.score(X_train_lin, y_train_lin))
print("Test", reg_lr.score(X_test_lin, y_test_lin))

Train 0.8069244730045397
Test 0.8284337301371468


In [28]:
scores_reg_lin = []

cv = KFold(n_splits = 10, random_state = 100, shuffle = True)

for train_index, test_index in cv.split(X_lin):
    
    X_train_lin = X_lin.iloc[train_index]
    X_test_lin = X_lin.iloc[test_index]
    y_train_lin = Y_lin.iloc[train_index]
    y_test_lin = Y_lin.iloc[test_index]
    
    
    reg_lr.fit(X_train_lin, y_train_lin)
    
    #y_pred_lin_train = reg_lr.predict(X_train_lin)
    y_pred_lin_test = reg_lr.predict(X_test_lin)
        
    scores_reg_lin.append(reg_lr.score(X_test_lin, y_test_lin))
print(' R^2: ')
print(scores_reg_lin)
print('\n', 'Mean: ', sum(scores_reg_lin)/len(scores_reg_lin))

 R^2: 
[0.8513120970444211, 0.8339413255516418, 0.7776170303361295, 0.8541116621620547, 0.8621138172069769, 0.4752975854291568, 0.8729020776732694, 0.6454456320287001, 0.8290924033036183, 0.8400634171071778]

 Mean:  0.7841897047843147


## Cтекинг

In [32]:
def get_meta_features(clf, X_train, y_train, X_test, stack_cv):
    
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(y_test_clf, dtype=float)
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        
        clf.fit(X_train.iloc[train_ind], y_train[train_ind])
        
        meta_train[test_ind] = clf.predict_proba(X_train.iloc[test_ind])[:, 1]
        
        meta_test += clf.predict_proba(X_test)[:, 1]
    
    return meta_train, meta_test / stack_cv.n_splits

In [33]:
from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(penalty='l1', C=0.1, n_jobs = -1)

clf_lr.fit(X_train_clf, y_train_clf)

D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l1',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
from sklearn.model_selection import StratifiedKFold

stack_cv = StratifiedKFold(n_splits=10, random_state=600)

meta_train = []
meta_test = []
col_names = []

print('LR features...')
meta_tr, meta_te = get_meta_features(clf_lr, X_train_clf, y_train_clf, X_test_clf, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('lr_pred')

print('RF features...')
meta_tr, meta_te = get_meta_features(clf_rf, X_train_clf, y_train_clf, X_test_clf, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('rf_pred')

LR features...


D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default 

RF features...


In [35]:
X_meta_train = pd.DataFrame(np.stack(meta_train, axis=1), columns=col_names)
X_meta_test = pd.DataFrame(np.stack(meta_test, axis=1), columns=col_names)

In [37]:
clf_lr_meta = LogisticRegression(penalty='l2', C=1)
clf_lr_meta.fit(X_meta_train, y_train_clf)

D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Program_Files\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [42]:
clf_lr_meta.score(X_meta_test, y_test_clf)

0.5616438356164384